# Data load 

In [2]:
# data is located in https://archive.ics.uci.edu/ml/datasets/dodgers+loop+sensor
trafficPath="file:///Users/chenwisely/notebook/Dodgers.data"
gamesPath="file:///Users/chenwisely/notebook/Dodgers.events"

In [3]:
traffic = sc.textFile(trafficPath)
traffic.take(10)
# data content is each five min time, last 5 min's cars 

['4/10/2005 0:00,-1',
 '4/10/2005 0:05,-1',
 '4/10/2005 0:10,-1',
 '4/10/2005 0:15,-1',
 '4/10/2005 0:20,-1',
 '4/10/2005 0:25,-1',
 '4/10/2005 0:30,-1',
 '4/10/2005 0:35,-1',
 '4/10/2005 0:40,-1',
 '4/10/2005 0:45,-1']

In [5]:
games = sc.textFile(gamesPath)
games.take(10)
# data content is date,startTime,endTime,audience,Oppoent,WinOrLoss

['04/12/05,13:10:00,16:23:00,55892,San Francisco,W 9-8�',
 '04/13/05,19:10:00,21:48:00,46514,San Francisco,W 4-1�',
 '04/15/05,19:40:00,21:48:00,51816,San Diego,W 4-0�',
 '04/16/05,19:10:00,21:52:00,54704,San Diego,W 8-3�',
 '04/17/05,13:10:00,15:31:00,53402,San Diego,W 6-0�',
 '04/25/05,19:10:00,21:33:00,36876,Arizona,L 4-2�',
 '04/26/05,19:10:00,22:00:00,44486,Arizona,L 3-2�',
 '04/27/05,19:10:00,22:17:00,54387,Arizona,L 6-3�',
 '04/29/05,19:40:00,22:01:00,40150,Colorado,W 6-3�',
 '04/30/05,19:10:00,21:45:00,54123,Colorado,W 6-2�']

# Parse data into RDD

In [9]:
from datetime import datetime
import csv
from io import StringIO

def parseTraffic(row):
    # parse traffic function
    DATE_FMT="%m/%d/%Y %H:%M"
    row=row.split(",")
    row[0] = datetime.strptime(row[0] , DATE_FMT)
    row[1] = int(row[1])
    return (row[0],row[1])

In [10]:
trafficParsed = traffic.map(parseTraffic)
trafficParsed.take(10)

[(datetime.datetime(2005, 4, 10, 0, 0), -1),
 (datetime.datetime(2005, 4, 10, 0, 5), -1),
 (datetime.datetime(2005, 4, 10, 0, 10), -1),
 (datetime.datetime(2005, 4, 10, 0, 15), -1),
 (datetime.datetime(2005, 4, 10, 0, 20), -1),
 (datetime.datetime(2005, 4, 10, 0, 25), -1),
 (datetime.datetime(2005, 4, 10, 0, 30), -1),
 (datetime.datetime(2005, 4, 10, 0, 35), -1),
 (datetime.datetime(2005, 4, 10, 0, 40), -1),
 (datetime.datetime(2005, 4, 10, 0, 45), -1)]

# Daily Traffic Trend

In [17]:
#filter the -1 traffic and sum up each date's traffic 
dailyTrend = trafficParsed.filter(lambda x:x[1] >0).map(lambda x: (x[0].date() , x[1]))\
                          .reduceByKey(lambda x,y: x+y)

In [18]:
dailyTrend.take(10)

[(datetime.date(2005, 4, 11), 5153),
 (datetime.date(2005, 4, 14), 6423),
 (datetime.date(2005, 4, 15), 6472),
 (datetime.date(2005, 4, 16), 6004),
 (datetime.date(2005, 4, 17), 5322),
 (datetime.date(2005, 4, 18), 5601),
 (datetime.date(2005, 4, 19), 6049),
 (datetime.date(2005, 4, 21), 5986),
 (datetime.date(2005, 4, 22), 6038),
 (datetime.date(2005, 4, 23), 5366)]

In [21]:
# -x[1] to specify descending order 
dailyTrend.sortBy(lambda x: -x[1]).take(10)

[(datetime.date(2005, 7, 28), 7661),
 (datetime.date(2005, 7, 29), 7499),
 (datetime.date(2005, 8, 12), 7287),
 (datetime.date(2005, 7, 27), 7238),
 (datetime.date(2005, 9, 23), 7175),
 (datetime.date(2005, 7, 26), 7163),
 (datetime.date(2005, 5, 20), 7119),
 (datetime.date(2005, 8, 11), 7110),
 (datetime.date(2005, 9, 8), 7107),
 (datetime.date(2005, 9, 7), 7082)]

# Join 

In [22]:
# Join = Inner Join
# Left Outer Join = all key from the left RDD are returned 
# Right Outer Join = all key from the right RDD are returned 

In [23]:
# Join with Games

def parseGames(row):
    # get (date,opponent)
    DATE_FMT="%m/%d/%y"
    row = row.split(",")
    row[0] = datetime.strptime(row[0] , DATE_FMT).date()
    return (row[0],row[4])

gamesParsed = games.map(parseGames)
gamesParsed.take(10)


[(datetime.date(2005, 4, 12), 'San Francisco'),
 (datetime.date(2005, 4, 13), 'San Francisco'),
 (datetime.date(2005, 4, 15), 'San Diego'),
 (datetime.date(2005, 4, 16), 'San Diego'),
 (datetime.date(2005, 4, 17), 'San Diego'),
 (datetime.date(2005, 4, 25), 'Arizona'),
 (datetime.date(2005, 4, 26), 'Arizona'),
 (datetime.date(2005, 4, 27), 'Arizona'),
 (datetime.date(2005, 4, 29), 'Colorado'),
 (datetime.date(2005, 4, 30), 'Colorado')]

In [28]:
# we want to keep traffic data no matter that day has game or not
# So use left outer join 
dailyTrendCombined = dailyTrend.leftOuterJoin(gamesParsed)
dailyTrendCombined.take(10)

[(datetime.date(2005, 4, 11), (5153, None)),
 (datetime.date(2005, 4, 15), (6472, 'San Diego')),
 (datetime.date(2005, 4, 17), (5322, 'San Diego')),
 (datetime.date(2005, 4, 19), (6049, None)),
 (datetime.date(2005, 4, 21), (5986, None)),
 (datetime.date(2005, 4, 22), (6038, None)),
 (datetime.date(2005, 4, 23), (5366, None)),
 (datetime.date(2005, 4, 24), (4319, None)),
 (datetime.date(2005, 4, 25), (6280, 'Arizona')),
 (datetime.date(2005, 4, 30), (6090, 'Colorado'))]

In [27]:
# inner join = only game day has data 
dailyTrendCombinedJoin = dailyTrend.join(gamesParsed)
dailyTrendCombinedJoin.take(10)

[(datetime.date(2005, 4, 15), (6472, 'San Diego')),
 (datetime.date(2005, 4, 17), (5322, 'San Diego')),
 (datetime.date(2005, 4, 25), (6280, 'Arizona')),
 (datetime.date(2005, 4, 30), (6090, 'Colorado')),
 (datetime.date(2005, 5, 1), (5178, 'Colorado')),
 (datetime.date(2005, 5, 4), (6423, 'Washington')),
 (datetime.date(2005, 5, 13), (6706, 'Atlanta')),
 (datetime.date(2005, 5, 21), (5767, 'LA Angels')),
 (datetime.date(2005, 6, 1), (6520, 'Chicago Cubs')),
 (datetime.date(2005, 6, 8), (6278, 'Detroit'))]

In [30]:
# right outer join = only hame day has data 
dailyTrendCombinedRightJoin = dailyTrend.rightOuterJoin(gamesParsed)
dailyTrendCombinedRightJoin.take(10)

[(datetime.date(2005, 4, 15), (6472, 'San Diego')),
 (datetime.date(2005, 4, 17), (5322, 'San Diego')),
 (datetime.date(2005, 4, 25), (6280, 'Arizona')),
 (datetime.date(2005, 4, 30), (6090, 'Colorado')),
 (datetime.date(2005, 5, 1), (5178, 'Colorado')),
 (datetime.date(2005, 5, 4), (6423, 'Washington')),
 (datetime.date(2005, 5, 13), (6706, 'Atlanta')),
 (datetime.date(2005, 5, 21), (5767, 'LA Angels')),
 (datetime.date(2005, 6, 1), (6520, 'Chicago Cubs')),
 (datetime.date(2005, 6, 8), (6278, 'Detroit'))]

In [32]:
def checkGameDay(row):
    #Date,Opponent,Type of Day,Number of Cars 
    if row[1][1] == None:
        return (row[0],row[1][1],"Regular Day",row[1][0])
    else:
        return (row[0],row[1][1],"Game Day",row[1][0])
    
dailyTrendByGames = dailyTrendCombined.map(checkGameDay)
dailyTrendByGames.take(10)

[(datetime.date(2005, 4, 11), None, 'Regular Day', 5153),
 (datetime.date(2005, 4, 15), 'San Diego', 'Game Day', 6472),
 (datetime.date(2005, 4, 17), 'San Diego', 'Game Day', 5322),
 (datetime.date(2005, 4, 19), None, 'Regular Day', 6049),
 (datetime.date(2005, 4, 21), None, 'Regular Day', 5986),
 (datetime.date(2005, 4, 22), None, 'Regular Day', 6038),
 (datetime.date(2005, 4, 23), None, 'Regular Day', 5366),
 (datetime.date(2005, 4, 24), None, 'Regular Day', 4319),
 (datetime.date(2005, 4, 25), 'Arizona', 'Game Day', 6280),
 (datetime.date(2005, 4, 30), 'Colorado', 'Game Day', 6090)]

In [33]:
# get the top 10 traffic day 
dailyTrendByGames.sortBy(lambda x:-x[3]).take(10)

[(datetime.date(2005, 7, 28), 'Cincinnati', 'Game Day', 7661),
 (datetime.date(2005, 7, 29), 'St. Louis', 'Game Day', 7499),
 (datetime.date(2005, 8, 12), 'NY Mets', 'Game Day', 7287),
 (datetime.date(2005, 7, 27), 'Cincinnati', 'Game Day', 7238),
 (datetime.date(2005, 9, 23), 'Pittsburgh', 'Game Day', 7175),
 (datetime.date(2005, 7, 26), 'Cincinnati', 'Game Day', 7163),
 (datetime.date(2005, 5, 20), 'LA Angels', 'Game Day', 7119),
 (datetime.date(2005, 8, 11), 'Philadelphia', 'Game Day', 7110),
 (datetime.date(2005, 9, 8), None, 'Regular Day', 7107),
 (datetime.date(2005, 9, 7), 'San Francisco', 'Game Day', 7082)]

# Avg value of each key

In [34]:
pairRDD= dailyTrendByGames.map(lambda x:(x[2],x[3]))
pairRDD.take(10)

[('Regular Day', 5153),
 ('Game Day', 6472),
 ('Game Day', 5322),
 ('Regular Day', 6049),
 ('Regular Day', 5986),
 ('Regular Day', 6038),
 ('Regular Day', 5366),
 ('Regular Day', 4319),
 ('Game Day', 6280),
 ('Game Day', 6090)]

In [40]:
combineByKeyResult = pairRDD.combineByKey(lambda value: (value,1),\
                                          lambda acc,value: (acc[0]+value,acc[1]+1),\
                                          lambda acc1,acc2: (acc1[0]+acc2[0],acc1[1]+acc2[1]))
combineByKeyResult.take(10)

[('Regular Day', (510656, 93)), ('Game Day', (482749, 81))]

In [41]:
combineByKeyResult.mapValues(lambda x:x[0]/x[1]).collect()

[('Regular Day', 5490.924731182796), ('Game Day', 5959.864197530864)]